In [1]:
import pandas as pd

In [2]:
all_features = ["tweet_id", "present_links", "tweet_timestamp", "engaging_user_id", "retweet_with_comment"]
train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [2, 4, 8, 14, 22], sep="\x01"
                   )

AttributeError: 'DataFrame' object has no attribute 'retweet_with_comment_timestamp'

In [3]:
train['retweet_with_comment_bool'] = train.retweet_with_comment.fillna(0)
train.loc[train.retweet_with_comment_bool != 0.0, 'retweet_with_comment_bool'] = 1.0
train['present_links_bool'] = train.present_links.fillna('0')

In [ ]:
train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()

In [ ]:
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100)))